## **Identify and analyze plant regions**

In [22]:
image_path = 'P1/D5/Stage 1/IMG_6749.jpg'

In [23]:
import cv2
import numpy as np

def analyze_plant_region(image_path):
    # Step 1: Image Acquisition
    image = cv2.imread(image_path)
    original_image = image.copy()

    # Step 2: Preprocessing
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    blurred_image = cv2.GaussianBlur(hsv_image, (5, 5), 0)

    # Step 3: Segmentation
    lower_green = np.array([30, 40, 40])  # Adjust these values as needed
    upper_green = np.array([90, 255, 255])
    mask = cv2.inRange(blurred_image, lower_green, upper_green)

    # Apply morphological operations to clean up the mask
    kernel = np.ones((5, 5), np.uint8)
    cleaned_mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    cleaned_mask = cv2.morphologyEx(cleaned_mask, cv2.MORPH_OPEN, kernel)

    # Step 4: Contour Detection
    contours, _ = cv2.findContours(cleaned_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Step 5: Feature Extraction and Analysis
    for contour in contours:
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        bounding_box = cv2.boundingRect(contour)
        x, y, w, h = bounding_box
        aspect_ratio = float(w) / h if h != 0 else 0
        
        # Filter out small contours or contours that are too wide or too tall
        if area < 500:  # Adjust this threshold based on your image
            continue
        if aspect_ratio < 0.2 or aspect_ratio > 5:  # Adjust aspect ratio limits as needed
            continue

        # Calculate centroid
        M = cv2.moments(contour)
        if M['m00'] != 0:
            centroid = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))
        else:
            centroid = (0, 0)

        # Optional: Print or log the features
        print(f"Area: {area:.2f}, Perimeter: {perimeter:.2f}, Aspect Ratio: {aspect_ratio:.2f}, Centroid: {centroid}")

        # Step 6: Visualization
        # Draw the contour as a border
        cv2.drawContours(original_image, [contour], -1, (0, 255, 0), 10)  # Green border with thickness 10
        cv2.putText(original_image, f'Area: {area:.2f}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 5, (255, 0, 0), 10)

    # Step 7: Output Results
    cv2.imshow('Original Image with Analyzed Plant Region', original_image)
    cv2.imshow('Mask', cleaned_mask)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage
analyze_plant_region(image_path)

Area: 194734.50, Perimeter: 8103.38, Aspect Ratio: 0.59, Centroid: (1397, 720)
